In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
AEdata = pd.read_csv("cleaned_AE_data.csv")

In [4]:
AEdata['Period'] = pd.to_datetime(AEdata['Period'])

In [5]:
AEdata = AEdata.set_index('Period')

In [6]:
Emergency_admissions = AEdata['Total Emergency Admissions']

In [7]:
## 1. Split main dataset into Train Validation and Hold subsets of data
## Train (70%, 84), Validation (20%, 24), Hold (10%, 12). Total rows 120

In [8]:
# Training dataset (70%, 84)
traindata = Emergency_admissions.iloc[0:84]

In [9]:
# Validation dataset (20%, 24)
valdata = Emergency_admissions.iloc[83:108]

In [10]:
# Hold dataset (10%, 12)
hold = Emergency_admissions[107:115]

In [11]:
traindataf = traindata.to_frame().apply(np.int64)

In [12]:
# Renmae variable now as a dataframe
traindataf.rename(columns={'Total Emergency Admissions':'Emadmissions'})

,Emadmissions
Period,
2010-08-01,425702
2010-09-01,424900
2010-10-01,436215
2010-11-01,429099
2010-12-01,452728
...,...
2017-03-01,509805
2017-04-01,471328
2017-05-01,505843


In [15]:
valdataf = valdata.to_frame().apply(np.int64)

In [16]:
valdataf.rename(columns={'Total Emergency Admissions':'Emadmission'})

,Emadmission
Period,
2017-07-01,499259
2017-08-01,486074
2017-09-01,486799
2017-10-01,513217
2017-11-01,513019
2017-12-01,520803
2018-01-01,526045
2018-02-01,476867
2018-03-01,526066


## 4 Data preparation for the RNN Neural Network

#### We need to scale the data using the MinMaxScaler() 

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
# The dataset will be scaled between 0  and 1 
# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0,1))

In [19]:
Traindata = scaler.fit_transform(traindataf)

In [20]:
flat_Traindata = Traindata.flatten()

In [21]:
Traindata_scaled = pd.Series(flat_Traindata,
                            index=traindataf.index)

In [22]:
total_lags = 5

###  4.3 Build Target and Features datasets based on Traindata_scaled dataset 

In [23]:
## TARGET train dataset Y_train 

In [24]:
Y_train = Traindata_scaled.iloc[5:,]

In [25]:
## FEATURES train dataset X_train This is the reversed dataframe 

In [26]:
def get_features(data,total_lags):
    col_list=[]
    for time_lag in range(total_lags,0, -1):
        Lag_i = pd.DataFrame(data.shift(time_lag+1,axis=0,fill_value=0))
        col_list.append(Lag_i)
        features_array = pd.concat(col_list,axis=1)
        trunc_dataset = features_array.iloc[total_lags:]
    return trunc_dataset

In [27]:
X_train_rev = get_features(Traindata_scaled,total_lags)

In [28]:
type(X_train_rev)

pandas.core.frame.DataFrame

In [29]:
X_train_array = np.array(X_train_rev)

In [30]:
X_train_array.shape

(79, 5)

In [31]:
## Copy dataset
X_train_array_MODEL = X_train_array[:,:,np.newaxis]

In [32]:
#### Check in your Features TRAIN dataset you have THREE dimensions now 

In [33]:
#### Check in your Features TRAIN dataset you have THREE dimensions now 
X_train_array[:,:,np.newaxis].shape

(79, 5, 1)

In [34]:
X_train_array_MODEL.shape

(79, 5, 1)

In [35]:
# 75 samples 
# 5 sequence lags
# 1 feature

## TARGET Validation dataset Y_val 

In [36]:
### Scale the validation dataset 

In [37]:
Valdata = scaler.fit_transform(valdataf)

In [38]:
#### 4.2 Once the dataset has been scaled we need to turn it back into a Series object for the Network model build

In [39]:
flat_Valdata = Valdata.flatten()

In [40]:
Valdata_scaled = pd.Series(flat_Valdata,
                            index=valdataf.index)

In [41]:
total_lags = 5

In [42]:
###  4.3 Build Target and Features datasets based on Valdata_scaled dataset 

In [43]:
## TARGET validation dataset Y_val

In [44]:
Y_val = Valdata_scaled.iloc[5:,]

In [45]:
## FEATURES validation dataset X_val This is the reversed features dataframe

In [46]:
## Reverse order defined by (total_lags,0,-1)

In [47]:
def get_features(data,total_lags):
    col_list=[]
    for time_lag in range(total_lags,0, -1):
        Lag_i = pd.DataFrame(data.shift(time_lag+1,axis=0,fill_value=0))
        col_list.append(Lag_i)
        features_array = pd.concat(col_list,axis=1)
        trunc_dataset = features_array.iloc[total_lags:]
    return trunc_dataset

In [48]:
X_val_rev = get_features(Valdata_scaled,total_lags)

In [49]:
X_val_array = np.array(X_val_rev)

In [50]:
X_val_array.shape

(20, 5)

#### Check in your Features VALIDATION dataset you have THREE dimensions now 

In [51]:
 X_val_array[:,:,np.newaxis].shape

(20, 5, 1)

In [52]:
## Copy dataset
X_val_array_MODEL = X_val_array[:,:,np.newaxis]

In [57]:
X_val_array_MODEL.shape

(20, 5, 1)

In [58]:
# 20 samples 
# 5 sequence lags 
# 1 feature

### 5.1 Specify NN architecture. Build the RNN (Sequence to Sequence Recurrent Neural Network) model

In [71]:
from tensorflow.keras.layers import SimpleRNN 

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [73]:
from tensorflow.keras.layers import TimeDistributed

In [74]:
# model4.add(Dense(1, activation="relu"))


### Time distributed layer: it wraps any layer and applies it at every step of the input sequence.

#### This technique is that the loss will contain a term for the output of the RNN at each ond every time step, not just the output at the last time step

In [ ]:
model5 = Sequential()
model5.add(SimpleRNN(5,input_shape=(5,1), activation='relu'))
model5.add(SimpleRNN(5,return_sequences=True))
model5.add(TimeDistributed(Dense(5)))

ValueError: Input 0 of layer simple_rnn_7 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 5]

# 5.1 Display model Summary

In [76]:
model5.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, None, 5)           35        
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, None, 5)           55        
_________________________________________________________________
time_distributed (TimeDistri (None, None, 5)           30        
Total params: 120
Trainable params: 120
Non-trainable params: 0
_________________________________________________________________


### 5.2 Compile the model

In [77]:
### 5.2 Compile the model
## Then we compile the  model we have just created
# Model compile step specifies the loss function and the optimizer to use 
model5.compile(loss='mean_squared_error',optimizer='adam')

In [80]:
X_train_array_MODEL.shape

(79, 5, 1)

In [82]:
Y_train.shape

(79,)

In [81]:
X_val_array_MODEL.shape

(20, 5, 1)

In [83]:
Y_val.shape

(20,)

### 5.3 Train and evaluate the model

In [79]:
history = model5.fit(X_train_array_MODEL, Y_train, epochs=1000,
                    validation_data=(X_val_array_MODEL,Y_val))

Train on 79 samples, validate on 20 samples
Epoch 1/1000


InvalidArgumentError: Incompatible shapes: [32,5,5] vs. [32,1]
	 [[{{node Adam/gradients/loss/time_distributed_loss/SquaredDifference_grad/BroadcastGradientArgs}}]]

### 5.4 Display training parameters

In [ ]:
#### When we run the fit() function, it creates a History object containing the training parameters (history.params).
### fit() : Training parameters (history.params)
### fit() : The list of epochs it went through (history.epoch)
##  fit() : It contains a dictionary (history.history) containing the loss and extra metrics it measured at the end of each epoch
pd.DataFrame(history.history).plot(figsize=(20,10))
plt.grid(True)
plt.gca().set_ylim(0,1) # This sets vertical range to [0-1]
plt.show()
